# Plagiarism Detection Project

In this notebook, you will be tasked with building a plagiarism detector that examines a text file and performs binary classification; labeling that file as either plagiarized or not, depending on how similar the text file *is* to a provided source text. 

This task will be broken down into a few discrete steps:

>1. Load in and explore the corpus.
2. Clean and pre-process the data.
3. Define metrics for comparing the similarity of a given text file and an original text, and extract similarity features.
4. Select "good" features, by analyzing the correlations between different features.
5. Train a classifier, on the features you selected in step 4, to perform binary classification: plagiarized or not!
6. Evaluate your classifier and answer some questions about your approach.

You'll be defining a few different similarity metrics, as outlined in [this paper](https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c412841_developing-a-corpus-of-plagiarised-short-answers/developing-a-corpus-of-plagiarised-short-answers.pdf), which should help you build a robust plagiarism detector!

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset. So, first, let's load in and look at the data for this project.


---

## Read in the Libraries and Files

The first step in working with any dataset is loading the data in and noting what information is included in the dataset. This is an important step in eventually working with this data, and knowing what kinds of features you have to work with as you transform and group the data!

This plagiarism dataset is made of multiple text files; each of these files has characteristics that are is summarized in a `.csv` file named `file_information.csv`, which we can read in using `pandas`.

If you are working locally, you'll need to download the necessary text and file data [at this link](https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c4147f9_data/data.zip). It's assumed that this data is extracted and in a folder `data/`.


> Each text file is associated with **one task** (task A-E) and **one category of plagiarism**, which you will see described as characters or strings in the following dataframe.

As you look at the data, you'll also notice a few other features that were recorded about the person generating the text and the task difficulty level. This dataset was created by Paul Clough (Information Studies) and Mark Stevenson (Computer Science), at the University of Sheffield. You can read all about the data collection and corpus, at [their university webpage](https://ir.shef.ac.uk/cloughie/resources/plagiarism_corpus.html). 

> **Citation for data**: Clough, P. and Stevenson, M. Developing A Corpus of Plagiarised Short Answers, Language Resources and Evaluation: Special Issue on Plagiarism and Authorship Analysis, In Press. [Download]

In [221]:
# import libraries
import pandas as pd
import numpy as np
import os

In [222]:
csv_file = 'data/file_information.csv'
plagiarism_df = pd.read_csv(csv_file)

# print out the first 10 rows of data info
plagiarism_df.head(10)

,File,Task,Category
0,g0pA_taska.txt,a,non
1,g0pA_taskb.txt,b,cut
2,g0pA_taskc.txt,c,light
3,g0pA_taskd.txt,d,heavy
4,g0pA_taske.txt,e,non
5,g0pB_taska.txt,a,non
6,g0pB_taskb.txt,b,non
7,g0pB_taskc.txt,c,cut
8,g0pB_taskd.txt,d,light
9,g0pB_taske.txt,e,heavy


## Types of Plagiarism

To detect cases of plagiarism, we are most interested in the `Task` and `Category` columns above.

###  Five task types, A-E
* Each text file contains an answer to one short answer question; these questions are labeled as tasks A-E.
* Each task, A-E, is about a topic that might be included in the Computer Science curriculum that was created by the authors of this dataset (cited below). 
* For each of these questions a set of answers were obtained using a variety of approaches, some of which simulate cases in which the answer *is plagiarized* and others that simulate the case in which the answer *is not plagiarized*.

### Four categories of plagiarism 

To simulate plagiarism, the creators used a Wikipedia entry as a source text to provide to participants. Then the participants were given one of the following levels of plagiarism:

1. `cut`: Participants were asked to answer the question by simply copying and pasting text from the relevant Wikipedia article.
2. `light`: Participants were asked to paraphrase and base their answer on text found in the Wikipedia article.
3. `heavy`: Participants were once again asked to base their answer on the relevant Wikipedia article but were instructed to rephrase the text to generate an answer with the same meaning as the source text, but expressed using different words and structure.
4. `non`: Participants were provided with learning materials in the form of either lecture notes or sections from textbooks that could be used to answer the relevant question. Participants were asked to read these materials and then attempt to answer the question using their *own* knowledge and research.
5. `orig`: This is a specific category for the original, source text. We will use these files only for comparison purposes.

> So, out of the submitted files, the only category that does not contain any plagiarism is `non`.

In the next cell, print out some stats about the data.

In [223]:
# print out some stats about the data
print('Number of files: ', plagiarism_df.shape[0])  # .shape[0] gives the rows 
# .unique() gives unique items in a specified column
print('Number of unique tasks/question types (A-E): ', (len(plagiarism_df['Task'].unique())))
print('Unique plagiarism categories: ', (plagiarism_df['Category'].unique()))

Number of files:  100
Number of unique tasks/question types (A-E):  5
Unique plagiarism categories:  ['non' 'cut' 'light' 'heavy' 'orig']


You should see the number of text files in the dataset as well as the name of each file and some associated characteristics. **Note that this *includes* the 5 _original_ wikipedia files for tasks A-E.** If you take a look at the `data_files` directory, you'll notice that the originals start with the filename `orig_` as opposed to `g` for "group." So, in total there are 100 files, 95 of which are answers (submitted by people) and 5 of which are the original source texts.

Your goal will be to use this information, and the contents of a given text file, to eventually classify any given text file into one of two categories, plagiarized or not-plagiarized, by comparing it to the original source text.

### Distribution of Data

Next, let's look at the distribution of data. In this course, we've talked about the importance of datasets in informing how you develop an algorithm. So, here, we'll ask: **how evenly is our data distributed among different tasks and plagiarism levels?**

For many machine learning algorithms, we want a large dataset that is fairly evenly distributed among the class we are interested in (in this case, text that is plagiarized or not). If we have a very small dataset or class imbalance, we'll have to be very careful about how we define a model, later on, to get the best results!

Below, you should notice two things:
* Our dataset is quite small, especially with respect to examples of varying plagiarism levels. This is one of the reasons why we've decided to make this a binary classification project; this way, we can group light, heavy and cut categories into one larger set of plagiarized examples.
* The data is distributed fairly evenly across task and plagiarism types.

In [224]:
# Show counts by different tasks and amounts of plagiarism

# group and count by task
counts_per_task=plagiarism_df.groupby(['Task']).size().reset_index(name="Counts")
print("\nTask:")
display(counts_per_task)

# group by plagiarism level
counts_per_category=plagiarism_df.groupby(['Category']).size().reset_index(name="Counts")
print("\nPlagiarism Levels:")
display(counts_per_category)

# group by task AND plagiarism level
counts_task_and_plagiarism=plagiarism_df.groupby(['Task', 'Category']).size().reset_index(name="Counts")
print("\nTask & Plagiarism Level Combos :")
display(counts_task_and_plagiarism)


Task:


,Task,Counts
0,a,20
1,b,20
2,c,20
3,d,20
4,e,20



Plagiarism Levels:


,Category,Counts
0,cut,19
1,heavy,19
2,light,19
3,non,38
4,orig,5



Task & Plagiarism Level Combos :


,Task,Category,Counts
0,a,cut,4
1,a,heavy,3
2,a,light,3
3,a,non,9
4,a,orig,1
5,b,cut,3
6,b,heavy,4
7,b,light,3
8,b,non,9
9,b,orig,1


---
## Pre-Process the Data

Now that you've explored the data a bit, you'll need to pre-process this data. These steps will mainly include converting categorical data (like plagiarism labels) into **numerical data** that we can use as input into a model.

In the next few cells, you'll be tasked with creating a new dataframe of desired information about all of the files in the `data_files/` directory. For each file, you'll want to keep track of the text that each file contains, the corresponding task (a-e), and the plagiarism level. Most of this data can be retrieved or calculated from the `.csv` file.

### EXERCISE: Convert categorical to numerical data

Complete the below function `clean_dataframe` that reads in a `.csv` file by name, and returns a *new* dataframe with numerical data. The new dataframe should have the following properties:

1. 3 columns: `File`, `Task`, `Category`. For the most part these contain the same information as can be found in the original `.csv` file, with a few exceptions.
2. Convert all `Category` labels to numerical labels according to the following rules:
    * 0 is no plagiarism, coded with Category 'non'.
    * 1 is that there was low use of the original file, coded with Category 'heavy' to indicate heavy revision of text from the original answer.
    * 2 is that there was medium use of the original file, coded with Category 'light' to indicate only light revision text from the original answer.
    * 3 is that there was heavy plagiarism of the original file, coded with Category 'cut' to indicate the answer was cut and pasted 'as-is' from original answer (no revision).
    * -1 indicates an original file, coded with Category `orig`.
3. Your function should return this cleaned dataframe of 100 rows.

### Tips for completing this exercise

Your complete code should correctly parse the `.csv` file that contains filenames and plagiarism levels.

After running your function, you should get a dataframe with rows that look something like the following: 
```
      File          Task  Category
0	g0pA_taska.txt	a	0
1	g0pA_taskb.txt	b	3
2	g0pA_taskc.txt	c	2
3	g0pA_taskd.txt	d	1
4	g0pA_taske.txt	e	0
...
```

In [225]:
def clean_dataframe(csv_file='data/file_information.csv'):
    '''This function reads in a dataframe and converts the 'Category' column
       from categorical to numerical data.
       :param csv_file: the directory for the csv file to be read in
       :return: a dataframe with numerical "Category" data'''
    # read in csv file
    # create a cdataframe that has columns for all the data in the csv file
    # and that replaces categorical with numerical data
    clean_df = pd.read_csv(csv_file) 
    cleanup_categories = {
        "Category": {
            "non": 0,
            "heavy": 1,
            "light": 2,
            "cut": 3,
            "orig": -1
        }
    }
    clean_df.replace(cleanup_categories, inplace=True)
    display(clean_df.head(10))
    return clean_df


### Test cell

Below is a test cell. The goal of a cell like this is to ensure that your code is working as expected, and to form any variables that might be used in later tests/code, in this case, the data frame `clean_df`.

These tests are not completely rigorous, but they are a great way to check that you are on the right track!

In [226]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import problem_unittests as tests

# test clean_dataframe function
tests.test_clean_df(clean_dataframe)

# if passed, create new `clean_df`
clean_df = clean_dataframe(csv_file ='data/file_information.csv')

# check work
clean_df.head(10)

,File,Task,Category
0,g0pB_taske.txt,e,1
1,g0pC_taska.txt,a,1
2,g0pC_taskb.txt,b,0
3,g0pC_taskc.txt,c,0
4,g0pC_taskd.txt,d,3
5,g0pC_taske.txt,e,2
6,g1pB_taskb.txt,b,0
7,g1pB_taskc.txt,c,1
8,g1pB_taskd.txt,d,2
9,g1pB_taske.txt,e,3


Tests Passed!


,File,Task,Category
0,g0pA_taska.txt,a,0
1,g0pA_taskb.txt,b,3
2,g0pA_taskc.txt,c,2
3,g0pA_taskd.txt,d,1
4,g0pA_taske.txt,e,0
5,g0pB_taska.txt,a,0
6,g0pB_taskb.txt,b,0
7,g0pB_taskc.txt,c,3
8,g0pB_taskd.txt,d,2
9,g0pB_taske.txt,e,1


,File,Task,Category
0,g0pA_taska.txt,a,0
1,g0pA_taskb.txt,b,3
2,g0pA_taskc.txt,c,2
3,g0pA_taskd.txt,d,1
4,g0pA_taske.txt,e,0
5,g0pB_taska.txt,a,0
6,g0pB_taskb.txt,b,0
7,g0pB_taskc.txt,c,3
8,g0pB_taskd.txt,d,2
9,g0pB_taske.txt,e,1


### EXERCISE: Pre-process text data & add useful columns to a dataframe

Next, you'll add some additional information to your dataframe. Recall that the end goal of this project is to look at the text in one file, compare it to an original file, and label that text as either _plagiarized_ or _not_. To complete this task, you'll need to add some information to your existing dataframe: 

1. Add a column `Text` that includes the text of each file; this should be lowercase text with any non-alphanumeric characters, and extraneous whitespace characters, removed.
2. Add a column `Class` that labels text as `0` not-plagiarized, `1` plagiarized, or `-1` original text; these values should be informed by the existing `Category` column.


### Tips for completing this exercise

You've been given some text pre-processing code that you can use to help you complete this function. 

After running your function, you should get a dataframe with rows that look something like the following: 
```
          File	   Task  Category	       Text	                                 Class
0	g0pA_taska.txt	a	   0	inheritance is a basic concept of object orien...	0
1	g0pA_taskb.txt	b	   3	pagerank is a link analysis algorithm used by ...	1
2	g0pA_taskc.txt	c	   2	the vector space model also called term vector...	1
3	g0pA_taskd.txt	d	   1	bayes theorem was names after rev thomas bayes...	1
...
95	orig_taska.txt	a	  -1	in object oriented programming inheritance is ...	-1
96	orig_taskb.txt	b	  -1	pagerank is a link analysis algorithm used by ...	-1
97	orig_taskc.txt	c	  -1	vector space model or term vector model is an ...	-1
...

```

### Text pre-processing helper function

The below helper function need not be changed, it takes in a file and returns an `all_text` lowercase result with some special characters and white spaces removed.

In [227]:
# helper function for pre-processing text given a file
import re

def process_file(file):
    # put text in all lower case letters 
    all_text = file.read().lower()
    # remove all non-alphanumeric chars
    all_text = re.sub(r"[^a-zA-Z0-9]", " ", all_text)
    # remove newlines/tabs, etc. so it's easier to match phrases, later
    all_text = re.sub("  ", " ", all_text)
    all_text = re.sub(r"\t", " ", all_text)
    all_text = re.sub(r"\n", " ", all_text)

    return all_text

### Add new columns `Text` and `Class` to the dataframe

This function assumes that you are passing in a `clean_df` as was created above, with a numerical Category column.

In [228]:
def complete_dataframe(clean_df, files, file_directory='data/'):
    '''Creates a dataframe with additional Text and Class columns.
       :param clean_df: a dataframe of file names, tasks, and numerical categories
       :param files: a list of files in a directory
       :param file_directory: the dir where our text files are stored, default='data/'
       :return: df, a dataframe holding the filename, task, category, text, and class
    '''
    
    # create a new dataframe with Text and Class columns
    complete_df = clean_df
    complete_df['Class'] = complete_df.apply(lambda row: 0 if row['Category'] == 0 else (-1 if row['Category'] == -1 else 1), axis = 1)
    for file in files:
        newFile = open('./data/' + file, 'r', errors="ignore")
        cleaned = process_file(newFile)
        complete_df.at[complete_df[complete_df['File'] == file].index.item(), 'Text'] = cleaned
    return complete_df


# complete_dataframe(clean_df, [file for file in os.listdir('data/processed') if file.endswith('.txt')])

In [229]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test complete_dataframe function
# params: clean_df from before, and complete_dataframe function
tests.test_complete_df(clean_df, complete_dataframe)

# if passed, create complete `df`
# Creates a list of filenames from 'txt' files in data/ directory
files = [file for file in os.listdir('data/') if file.endswith('.txt')]

# Calls function from above, passing in clean_df from previous step, gets new dataframe
df = complete_dataframe(clean_df, files)

# check results
df.head(10)

Tests Passed!


,File,Task,Category,Class,Text
0,g0pA_taska.txt,a,0,0,inheritance is a basic concept of object orien...
1,g0pA_taskb.txt,b,3,1,pagerank is a link analysis algorithm used by ...
2,g0pA_taskc.txt,c,2,1,the vector space model also called term vector...
3,g0pA_taskd.txt,d,1,1,bayes theorem was names after rev thomas bayes...
4,g0pA_taske.txt,e,0,0,dynamic programming is an algorithm design tec...
5,g0pB_taska.txt,a,0,0,inheritance is a basic concept in object orien...
6,g0pB_taskb.txt,b,0,0,pagerank pr refers to both the concept and the...
7,g0pB_taskc.txt,c,3,1,vector space model is an algebraic model for r...
8,g0pB_taskd.txt,d,2,1,bayes theorem relates the conditional and marg...
9,g0pB_taske.txt,e,1,1,dynamic programming is a method for solving ma...


You can also read the full, processed answer text by printing out the `Text` at a certain index in the dataframe. You should see a resultant text that is all lowercase and without punctuation; this will make the texts easier to compare and analyze, later on!

In [230]:
# take a look at an example answer Text
test_index = 0 # first file/row

print(df['Text'][test_index])

inheritance is a basic concept of object oriented programming where the basic idea is to create new classes that add extra detail to existing classes this is done by allowing the new classes to reuse the methods and variables of the existing classes and new methods and classes are added to specialise the new class inheritance models the is kind of relationship between entities or objects  for example postgraduates and undergraduates are both kinds of student this kind of relationship can be visualised as a tree structure where student would be the more general root node and both postgraduate and undergraduate would be more specialised extensions of the student node or the child nodes  in this relationship student would be known as the superclass or parent class whereas  postgraduate would be known as the subclass or child class because the postgraduate class extends the student class  inheritance can occur on several layers where if visualised would display a larger tree structure for 

---

## Split data into training and test for modeling

The next cell will adds a column (*Datatype*) to a given dataframe to indicate if the record is: 
* `orig` - The tasks original answer from wikipedia.
* `train` - Training data for model training.
* `test` - Testing data for model evaluation.

The method below uses a helper function which you can view in the `helpers.py` file in the main project directory. This implements [stratified random sampling](https://en.wikipedia.org/wiki/Stratified_sampling) to randomly split data by task & plagiarism amount. Approximately 26% of the data is held out for testing and 74% of the data is used for training.

### Stratified sampling

The function **train_test_dataframe** splits the data in a dataframe using stratified sampling on plagiarism amount(0,1-3). It takes in a datframe that it assumes has `Task` and `Category` columns, and, returns a modified frame that indicates which Datatype a file falls into; this sampling will change slightly based on a passed in *random_seed*. Due to small sample size, this stratified random sampling will provides more stable results for a binary plagiarism classifier. Stability here is smaller *variance* in the accuracy of classifier, given a random seed.

### Locating a task's original (wiki) answer

After adding a `Datatype` column, I'll also add one more column, which will be helpful for comparing a submitted answer to an original source text.

A call to `helpers.add_orig_loc(df)` will add a column `Orig_idx` to the passed in dataframe to indicate the index (row) of the task's original answer (from wikipedia). This will be helpful in creating features that rely on comparing a submitted and source text. 

In [231]:
random_seed = 1 # can change; set for reproducibility

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import helpers

# create new df with Datatype (train, test, orig) column
df = helpers.train_test_dataframe(df, random_seed=random_seed)
# add Orig_idx column; this modifies df directly
helpers.add_orig_loc(df)

# check results
df.head(10)



Original Task Indices A-E: [95, 96, 97, 98, 99] 



,File,Task,Category,Class,Text,Datatype,Orig_idx
0,g0pA_taska.txt,a,0,0,inheritance is a basic concept of object orien...,train,95
1,g0pA_taskb.txt,b,3,1,pagerank is a link analysis algorithm used by ...,test,96
2,g0pA_taskc.txt,c,2,1,the vector space model also called term vector...,train,97
3,g0pA_taskd.txt,d,1,1,bayes theorem was names after rev thomas bayes...,train,98
4,g0pA_taske.txt,e,0,0,dynamic programming is an algorithm design tec...,train,99
5,g0pB_taska.txt,a,0,0,inheritance is a basic concept in object orien...,train,95
6,g0pB_taskb.txt,b,0,0,pagerank pr refers to both the concept and the...,train,96
7,g0pB_taskc.txt,c,3,1,vector space model is an algebraic model for r...,test,97
8,g0pB_taskd.txt,d,2,1,bayes theorem relates the conditional and marg...,train,98
9,g0pB_taske.txt,e,1,1,dynamic programming is a method for solving ma...,test,99


---
# Determining Plagiarism

Now that you've prepared this data, you can move on to the task of determining whether or not a given file is plagiarized or not. Let's review our goal:
> **Goal**: Build a model to classify a given text file as plagiarized or not.

This is a binary classification task and now you have all the data you need to build this! The `df` created above has a list of all our data files, the processed text within each file, and the task and class label for that file (0 = not plagiarized, 1 = plagiarized). 

For example, say you want to train a model to determine if the first file `g0pA_taska.txt` is plagiarized (1) or not (0). You'll first want to look at the text of that file *and* the corresponding original source text file; this is a task A file and so you'll want to compare it to the text of `orig_taska.txt`. You can extract features that measure the similarity of these two texts and feed those into a model that will classify `g0pA_taska.txt` as plagiarized or not.

To build this model, the rest of this project is broken down into exercises that will be all about:
* Creating relevant, plagiarism-detection features
* Defining a model that is trained on the features and the training data/labels
* Evaluating the model on our test data

It *is* assumed, for ease of testing, that you will not get rid of or rename the original column values and names from above [File, Task, Category, Text, Class, Datatype, Orig_idx]. But, as you go, you're encouraged to develop as many helper functions or *additional* dataframes/columns, as you see fit.

---
# Similarity Metrics 

One of the ways we might go about detecting plagiarism, is by computing **similarity metrics** that measure how similar a given text answer is as compared to the original wikipedia answer (for a specific task, a-e). The similarity metrics we will use are informed by the [plagiarism paper](https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c412841_developing-a-corpus-of-plagiarised-short-answers/developing-a-corpus-of-plagiarised-short-answers.pdf) that is also linked at the top of this notebook. In this paper, researchers created features called **containment** and **longest common subsequence**. 

Using these features as input, we will train a model (naive bayes or logistic) to distinguish between plagiarized and valid text files. Then we will evaluate the accuracy of the model by testing it on our test data.

## Feature Engineering

Let's talk a bit more about what features we want to include in a plagiarism detection model and to calculate such features. In the following explanations, I'll refer to a submitted text file as a **Student Task Answer Text** and the original, wikipedia source file (that we want to compare that answer to) as the **Wikipedia Task Text**.

### Containment

Your first task will be to create **containment features**.

> Containment is defined as the **intersection** of the n-gram word count of the Wikipedia Task Text with the n-gram word count of the Student Task Answer Text *divided* by the n-gram word count of the Student Task Answer Text. In other words, this is a measure, between 0 and 1, of how many n-grams these two texts have in common; you can calculate this value using count or tf-idf vectorization.

In this project, it will be up to you to decide on the appropriate `n` or several useful `n`'s to use in your final model.

### EXERCISE: Create containment features

Given the dataframe that you've created, you should have all the information you need to compare any Student Task Answer Text with its appropriate Wikipedia Task Text (the source text). An answer for task A should be compared to the source text for task A, just as answers to task's B, C, D, and E should be compared to the corresponding original source text.

In this exercise, you'll complete the function, `calculate_containment` which calculates containment based upon an passed in dataframe (such as the one you created above with all the text and task information), an input *n-gram* size (a positive integer), which is used to create a specific n-gram containment value, and a file_index, which specifies the row number of the file we are interested in calculating the containment value of.

### Tips for completing this exercise

The general steps to complete this function are as follows, given a passed in dataframe and n-gram length like 1, 2, 3, etc.
1. Calculate n-gram counts for every one of our 100 files. It is suggested that you use a [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) or [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) that can count n-grams when applied to a text.
2. For the specified Student Task Answer Text (as specified by `file_index`, compare it's n-gram count to the corresponding Wikipedia Task Text n-gram count to calculate the containment, according to the equation below: the intersection of the source and student n-gram counts *divided* by the student n-gram count.

    >$$ \frac{\text{n-gram}_{student} \cap \text{n-gram}_{original}}{\text{n-gram}_{student}} $$
    
3. Return the containment value for that file.

To complete this `calculate_containment` function, you may define additional, helper functions in this notebook as you see fit!

In [232]:
# text_row = df.loc[79]
# print(text_row['File'])
# print(os.path.splitext(os.path.basename(text_row['File']))[0].split('_')[1])

# task_original = {
#     'taska': 95,
#     'taskb': 96,
#     'taskc': 97,
#     'taskd': 98,
#     'taske': 99
# }

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import os

def return_ngram(n, data):
    tfid = TfidfVectorizer(stop_words=None, ngram_range=(n, n))
    count_vectorizer = CountVectorizer(stop_words=None, ngram_range=(n, n))
    count_vectorizer.fit_transform([data['Text']])
    tfid.fit_transform([data['Text']])
    return set(count_vectorizer.get_feature_names())

# Function that calculates containment for ALL files in df, for specified n-grams
# returns containment value for the file in passed in `file_index` row
def calculate_containment(df, n, file_index):
    ''' Calculates the containment for specified n-grams and for a file in
       a passed in `file_index` row.
       :param df: complete dataframe with rows [File, Task, Categry, Text, Class]
       :param n: n for defining the length of an n-gram count
       :param file_index: the index for the file this function should analyze.
       :return: containment value for the file located at `file_index`.'''

    # create a matrix of ngram counts (iterate over all text files)
    # calculate the the intersection of ngram counts between a student answer and original text
    # return the normalized containment value
    submission = return_ngram(n, df.loc[file_index])
    original = return_ngram(n, df.loc[df.loc[file_index]['Orig_idx']])

    return len(submission.intersection(original)) / len(submission)

In [ ]:
# Import pipeline
from sklearn.pipeline import Pipeline

# Import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Import other preprocessing modules
from sklearn.preprocessing import Imputer
from sklearn.feature_selection import chi2, SelectKBest

# Select 300 best features
chi_k = 300

# Import functional utilities
from sklearn.preprocessing import FunctionTransformer, MaxAbsScaler
from sklearn.pipeline import FeatureUnion

# Perform preprocessing
get_text_data = FunctionTransformer(combine_text_columns, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC_COLUMNS], validate=False)

# Create the token pattern: TOKENS_ALPHANUMERIC
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Instantiate pipeline: pl
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer(token_pattern=TOKENS_ALPHANUMERIC,ngram_range=(1, 2))),
                    ('dim_red', SelectKBest(chi2, chi_k))
                ]))
             ]
        )),
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

### Testing your implementation

After you've implemented this function, you can test out its behavior. 


In [244]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test containment calculation
# params: complete_df from before, and containment function
tests.test_containment(df, calculate_containment)


Tests Passed!


**See the results for yourself!**

The below code iterates through the first few files, and calculates the containment values for a specified n.

>If you've implemented this correctly, you should see that the non-plagiarized have low or close to 0 containment values and that plagiarized examples have higher containment values (higher for more severe, cut-and-paste-like cases). You may change the value of n and you should generally see higher containment values for smaller values of n.

I recommend applying your code to multiple files and comparing the resultant containment values. You should see that the highest containment values correspond to files with the highest category (3) of plagiarism level.

In [245]:
# see results for yourself!
# do higher containment values correspond to higher plagiarism levels?

test_indices = range(5) # first few files
n = 3

for i in test_indices:
    print('Category of plagiarism (3 = highest level, cut): ', df.loc[i, 'Category'])
    test_val = calculate_containment(df, n, file_index=i)
    print(str(n)+ '-gram containment value: '+ str(test_val))
    print()


Category of plagiarism (3 = highest level, cut):  0
3-gram containment value: 0.00975609756097561

Category of plagiarism (3 = highest level, cut):  3
3-gram containment value: 0.9635416666666666

Category of plagiarism (3 = highest level, cut):  2
3-gram containment value: 0.6084905660377359

Category of plagiarism (3 = highest level, cut):  1
3-gram containment value: 0.15934065934065933

Category of plagiarism (3 = highest level, cut):  0
3-gram containment value: 0.032432432432432434



### QUESTION 1: Does containment calculated using TF-IDF vectorization give same containment value as that calculated using count vectorization? Why or why not?


**Answer:**

The same containment values were observed using TF-IDF vectorization as count vectorization.

### QUESTION 2: Why can we calculate features across *all* data (training & test), prior to splitting the dataframe for modeling? That is, what about the calculated features makes it so the test and training data do not influence each other?

**Answer:**



### Longest Common Subsequence

Containment a good way to find overlap in word usage between two documents; it may help identify cases of cut-and-paste as well as paraphrased levels of plagiarism. Since plagiarism is a fairly complex task with varying levels, it's often useful to include other measures of similarity. The paper also discusses a feature called **longest common subsequence**.

> The longest common subsequence is the longest string of words (or letters) that are *the same* between the Wikipedia Task Text and the Student Task Answer. This value is also normalized by dividing by the total number of words (or letters) in the  Student Task Answer. 

In this exercise, we'll ask you to calculate the longest common subsequence of words.

### EXERCISE: Calculate the longest common subsequence

Complete the function `lcs_norm_word`; this should calculate the *longest common subsequence* of words between a Student Task Answer Text and corresponding Wikipedia Task Text. 




### Tips to complete this exercise

This algorithm depends on looking at two texts and comparing them word by word. You can solve this problem in multiple ways. First, it may be useful to `.split()` each text into lists of comma separated words to compare. Then, you can iterate through each word in the texts and compare them, adding to your value for LCS as you go. 

The method I recommend for implementing an efficient LCS algorithm is: using a matrix and dynamic programming. **Dynamic programming** is all about breaking a larger problem into a smaller set of subproblems, and building up a complete result without having to repeat any subproblems. 

This approach assumes that you can split up a large LCS task into a combination of smaller LCS tasks. Let's look at a simple example that compares letters:

* S = "ABCD"
* O = "BD"

We can see right away that the longest subsequence of _letters_ here is 2 (B and D are in sequence in both strings). And we can calculate this by looking at relationships between each letter in the two strings, S and O. You can also use a matrix to find the LCS.

### The matrix rules

You can break up an LCS task into a series of smaller tasks and efficiently fill up an LCS matrix one cell at a time. Each grid cell only depends on the values in the grid cells that are directly on top and to the left of it, or on the diagonal/top-left. The rules are as follows:
* Start with a matrix that has one extra row and column of zeros.
* As you traverse your string:
    * If there is a match, fill that grid cell with the value to the top-left of that cell *plus* one. So, in our case, when we found a matching B-B, we added +1 to the value in the top-left of the matching cell, 0.
    * If there is not a match, take the *maximum* value from either directly to the left or the top cell, and carry that value over to the non-match cell.

<img src='notebook_ims/matrix_rules.png' width=40% />

* After completely filling the matrix, **the bottom-right cell will hold the non-normalized LCS value**.

This matrix treatment can be applied to a set of words instead of letters. Your function should apply this to the words in two texts and return the normalized LCS value.

In [246]:
import collections

def memoize(fn):
    def inner(*args):
        try:
            return cache[args]
        except IndexError:
            cache[args] = fn(*args)
            return cache[args]
    return inner


def lcs_norm_word(source_text, answer_text):
    '''Calculates the normalized LCS between a source and answer text.
       :param source_text: pre-processed Wikipedia source text
       :param answer_text: pre-processed submitted answer text
       :return: normalized LCS value between the two texts.'''
    
    # Split texts into lists of words
    # Calculate normalized LCS
#     lcs_norm = None
    
#     return lcs_norm
    # https://www.geeksforgeeks.org/longest-common-subsequence-dp-4/
    source_text = source_text.split()
    answer_text = answer_text.split()
    m = len(source_text) 
    n = len(answer_text) 
  
    matrix = [[None]*(n+1) for i in range(m+1)] 
  
    for i in range(m + 1): 
        for j in range(n + 1): 
            if i == 0 or j == 0 : 
                matrix[i][j] = 0
            elif source_text[i - 1] == answer_text[j - 1]: 
                matrix[i][j] = matrix[i - 1][j - 1] + 1
            else: 
                matrix[i][j] = max(matrix[i - 1][j] , matrix[i][j - 1]) 
  
    # matrix[m][n] contains the length of matrixCS of source_text[0..n-1] & answer_text[0..m-1] 
    return matrix[m][n] / n

### Testing your implementation 

Let's start by testing out your code on the example given in the initial description.

In the below cell, we have specified strings S (submitted answer) and O (original source text). We know that these texts have 20 words in common and the submitted answer is 27 words long, so the normalized, longest common subsequence should be 20/27.


In [247]:
# Run the test scenario from above
# does your function return the expected value?

S = "i think pagerank is a link analysis algorithm used by google that uses a system of weights attached to each element of a hyperlinked set of documents"
O = "pagerank is a link analysis algorithm used by the google internet search engine that assigns a numerical weighting to each element of a hyperlinked set of documents"


lcs = lcs_norm_word(O, S)


###
assert lcs==20/27., "Incorrect LCS value, expected about 0.7408, got "+str(lcs)

print('Test passed!')
print('LCS = ', lcs)

Test passed!
LCS =  0.7407407407407407


This next cell runs a more rigorous test.

In [248]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test lcs implementation
# params: complete_df from before, and lcs_norm_word function
tests.test_lcs(df, lcs_norm_word)


Tests Passed!


Finally, take a look at a few resultant values for `lcs_norm_word`. Just like before, you should see that higher values correspond to higher levels of plagiarism.

In [249]:
# test on your own
test_df = df.copy()

test_range = range(5) # look at first few rows, can change this range

# iterate through first few docs and calculate LCS
for test_index in test_range:
    print('Category (0-3): ', test_df.loc[test_index, 'Category'])
    
    # calculate lcs
    lcs_val = lcs_norm_word(test_df.loc[test_df.loc[test_index,'Orig_idx'], 'Text'], 
                            df.loc[test_index, 'Text'])
    print(lcs_val)
    print()
    

Category (0-3):  0
0.1917808219178082

Category (0-3):  3
0.8207547169811321

Category (0-3):  2
0.8464912280701754

Category (0-3):  1
0.3160621761658031

Category (0-3):  0
0.24257425742574257



---
# Create features & determines models to test

Now that you've completed the feature calculation functions, it's time to actually create multiple features and decide on which ones to use in your final model!

In the below cells, you're given two helper functions to help you create ,multiple features and add the values as additional columns to an existing dataframe. These functions will be used to create the features you want to use in your final model; it will be up to you to decide on how many containment features to use and whether or not to use least common subsequence as a feature in your model.

### Creating multiple containment features

Your completed `calculate_containment` function will be called in the next cell, which defines the helper function `create_containment_features`. 

**This function adds containment features to an existing, passed in dataframe `df` based upon the passed in arguments.** This gives you the ability to easily create several containment features of different n-gram lengths.

In [250]:
## Do not need to change ##
# Function creates containment features for all files in a df and adds them to the given dataframe
def create_containment_features(df, n, column_name):
    
    # iterates through dataframe rows
    for index in df.index:
        # Computes features using function above for *answers* (not original files)
        if df.loc[index,'Category'] > -1:
            df.loc[index, column_name] = calculate_containment(df, n, index)
        # Sets value to -1 for original tasks 
        else:
            df.loc[index, column_name] = -1
            
    # returns nothing because dataframe is directly modifies
    print(str(n) + '-gram containment features created!')


### Creating LCS features

Below, your complete function is used to create LCS features and add them as columns to an existing dataframe.


In [251]:
## Do not need to change ##
# Function creates lcs feature and add it to the dataframe
def create_lcs_features(df, column_name='lcs_word'):
    
    # iterate through files in dataframe
    for index in df.index:
        
        # Computes LCS_norm words feature using function above for answer tasks
        if df.loc[index,'Category'] > -1:
            df.loc[index, column_name] = lcs_norm_word(df.loc[df.loc[index,'Orig_idx'], 'Text'],
                                                       df.loc[index, 'Text'])
        # Sets to -1 for original tasks 
        else:
            df.loc[index, column_name] = -1

    # nothing is returned, dataframe is directly modified
    print('LCS features created!')


## EXERCISE: Create features

The paper suggests calculating the following features: containment *1-gram to 5-gram* and *longest common subsequence*. 
> In this exercise, we suggest that you increase the n-gram range from *1-gram to 7-gram* as well as calculate *longest common subsequence*. 

This will give you 8 different features to choose from; defining and comparing 8 different features allows you to discard any features that seem redundant or bad, and choose to use the best features for your final model!

In the below cell **define an n-gram range**; these will be the n's you use to create n-gram containment features. The rest of the feature creation code is provided.

In [252]:
# Select ngram range, ex. [1, 3, 5]
ngram_range = [i % 2 != 0  for i in range(1, 5)]

# following code may take a moment to run
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
features_list = []

# Create features in a features_df
features_df = df.copy()

# Calculate features for containment for ngrams in range
for n in ngram_range:
    column_name = 'c_'+str(n)
    features_list.append(column_name)
    # create containment features
    create_containment_features(features_df, n, column_name)

# Calculate features for LCS_Norm Words 
features_list.append('lcs_word')
create_lcs_features(features_df, 'lcs_word')

# Check Dataframe
print()
print('Features: ', features_list)
features_df.head(5)

True-gram containment features created!
False-gram containment features created!
True-gram containment features created!
False-gram containment features created!
LCS features created!

Features:  ['c_True', 'c_False', 'c_True', 'c_False', 'lcs_word']


,File,Task,Category,Class,Text,Datatype,Orig_idx,c_True,c_False,lcs_word
0,g0pA_taska.txt,a,0,0,inheritance is a basic concept of object orien...,train,95,0.361702,1.0,0.191781
1,g0pA_taskb.txt,b,3,1,pagerank is a link analysis algorithm used by ...,test,96,1.000000,1.0,0.820755
2,g0pA_taskc.txt,c,2,1,the vector space model also called term vector...,train,97,0.848739,1.0,0.846491
3,g0pA_taskd.txt,d,1,1,bayes theorem was names after rev thomas bayes...,train,98,0.522523,1.0,0.316062
4,g0pA_taske.txt,e,0,0,dynamic programming is an algorithm design tec...,train,99,0.435644,1.0,0.242574


In [253]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# Get *only* the features
features_only = features_df[features_list].copy()

features_only.head(10)

,c_True,c_False,c_True,c_False,lcs_word
0,0.361702,1.0,0.361702,1.0,0.191781
1,1.000000,1.0,1.000000,1.0,0.820755
2,0.848739,1.0,0.848739,1.0,0.846491
3,0.522523,1.0,0.522523,1.0,0.316062
4,0.435644,1.0,0.435644,1.0,0.242574
5,0.379630,1.0,0.379630,1.0,0.161172
6,0.504505,1.0,0.504505,1.0,0.301653
7,0.824675,1.0,0.824675,1.0,0.621711
8,0.650000,1.0,0.650000,1.0,0.484305
9,0.847458,1.0,0.847458,1.0,0.597458


## Correlated Features

You should use feature correlation across the *entire* dataset to determine which features are ***too*** **highly correlated** with each other to include both features in a single model. For this analysis, you can use the *entire* dataset due to the small sample size we have. 

All of our features try to measure the similarity between two texts. Since our features are designed to measure similarity, it is expected that these features will be highly correlated. Generally, a Naive Bayes classifier relies on the assumption that features are *not* highly correlated; highly correlated features may ***over inflate the importance*** of a single feature. 

In [254]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""

# Create correlation matrix for just Features to determine different models to test
corr_matrix = features_only.corr().abs().round(2)

# display shows all of a dataframe
display(corr_matrix)

,c_True,c_False,c_True,c_False,lcs_word
c_True,1.00,0.84,1.00,0.84,0.98
c_False,0.84,1.00,0.84,1.00,0.78
c_True,1.00,0.84,1.00,0.84,0.98
c_False,0.84,1.00,0.84,1.00,0.78
lcs_word,0.98,0.78,0.98,0.78,1.00


### EXERCISE: Selecting "good" features by correlation value

Complete the `select_features` function below. This function should take in a dataframe that includes all our data and the extracted features, and a list of features that you want to include in your training/test datasets. You may assume that feature_list takes the form of a list of column names, ex. `['c_1', 'lcs_word']`.

* You will have to decide on a **cutoff** value for features that are *too* highly correlated.
* If you cannot find features that are less correlated than some cutoff value, it is suggested that you increase the number of features (longer n-grams) to choose from or simply use *only one or two* features in your final model to avoid highly correlated features.
* Your function should return two tuples:
    * `(train_x, train_y)`, a dataframe of training features and a list of corresponding class labels (0/1)
    * `(test_x, test_y)`, a dataframe of test features and a list of corresponding class labels (0/1)

Recall that our complete dataframe has a Datatype column that indicates whether data should be `train` or `test` data.

In [255]:
def select_features(features_df, feature_list):
    '''Creates training and test data based on selected features.
       :param features_df: dataframe with all fies, datatypes, and features
       :param feature_list: a list of feature names to select, ex ['c_1', 'lcs_word']
       :return: two tuples (train_x, train_y) and (text_x, test_y) of features and class labels.'''
    
    train_x = features_df[features_df["Datatype"] == "train"][features_list]
    train_y = np.array(features_df[features_df["Datatype"] == 'train']["Class"])
    
    test_x = features_df[features_df["Datatype"] == "test"][features_list]
    test_y = np.array(features_df[features_df["Datatype"] == 'test']["Class"])
    
    return (train_x, train_y), (test_x, test_y)
    

In [256]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that feature selection returns the correct datatypes
# params: features_df from a few cells above, and select_features function
tests.test_selection(features_df, select_features)


Tests Passed!


After completing your function code, it's time to define a list of features that you want to include in your final model. Write a feature_list, below.

In [257]:
# Select your best, low-correlated features
# feature_list should be a list of feature names, ex: ['c_1`, 'c_2', 'lcs_word']
feature_list = ['c_1', 'c_4', 'lcs_word']

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
(train_x, train_y), (test_x, test_y) = select_features(features_df, feature_list)

### Question 3: How did you decide on which features to include in your final model? 

( _the commented suggestions were used; real-world solution below._ )

Plagiarized texts can have language which is similar but not necessarily identical with the source.

2 possible ways to select features:
1. Alzahrani Similarity - http://sip04.softcomputing.net/smcc2011.pdf
2. Word2vec similarity, representing words in high-dimensional vector spaces - https://code.google.com/archive/p/word2vec/

---

## Modeling

You've decided on which features to use and gotten your training and test data in order. Finally, it's time to define and evaluate a model!

We suggest you use either a logistic or a naive bayes classifier to complete this binary, plagiarism classification task. There are some options to explore, and it will be up to you to test out a variety of models and choose the best one.
 

### EXERCISE: Train and evaluate a model

For this exercise, complete the following function `model_acc`. This should take in a few parameters:
* **model**: the model to test, such as a MultinomialNB(), etc.
* **train_x, train_y**: the training features and class labels (0/1), respectively
* **test_x, test_y**: the test features and class labels (0/1), respectively

### Metrics

Your function should train the model on the training data and evaluate it on the test data. 

In this case, we care about accuracy as well as the number of false positives and false negatives. We *do not* want to miss any plagiarized cases and we don't want to have any false positives either! 

In this vein, you function should return both the accuracy and a confusion matrix.

> Your model will ultimately be evaluated on its test accuracy; you should be able to consistently get over **90%** accuracy on the test data. 

You'll also be asked to justify your choice of model.

In [261]:
# Import Naive Bayes Classifier (using gaussian because features are continuous values)
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import metrics

def model_metrics(model, train_x, train_y, test_x, test_y):
    '''Trains a model and returns the accuracy and confusion matrix.
       :param model: model to train, ex. MultinomialNB()
       :param train_x, train_y: training features and class labels
       :param test_x, test_y: test features and class labels'''

    # Train the model with the training data
    # Create predictions on test data and calculate desired metrics
    model.fit(train_x, train_y)
    
    accuracy = metrics.balanced_accuracy_score(test_y, model.predict(test_x))
    confusion_matrix = metrics.confusion_matrix(test_y, model.predict(test_x))
    
    return accuracy, confusion_matrix


In [262]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that feature selection returns the correct datatypes
# params: features_df from a few cells above, and select_features function
tests.test_model_metrics(model_metrics)


Tests Passed!


In [267]:
# compare at least two models
# print out stats/results

## YOUR CODE HERE
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import SGDClassifier
# from sklearn.linear_model.logistic import LogisticRegression

data = [
    ['MultinomialNB', model_metrics(MultinomialNB(), train_x, train_y, test_x, test_y)],
    ['BernoulliNB', model_metrics(BernoulliNB(), train_x, train_y, test_x, test_y)],
    ['GaussianNB', model_metrics(GaussianNB(), train_x, train_y, test_x, test_y)],
# ['SGD Classifier', model_metrics(SGDClassifier(tol=1e-3, max_iter=1000), train_x, train_y, test_x, test_y)],
#     ['AdaBoost Classifier', model_metrics(AdaBoostClassifier(), train_x, train_y, test_x, test_y)],
#     ['Random Forest Classifier', model_metrics(RandomForestClassifier(n_estimators=100), train_x, train_y, test_x, test_y)],
#     ['K-Neighbors Classifier', model_metrics(KNeighborsClassifier(), train_x, train_y, test_x, test_y)],
]

pd.DataFrame(data, columns=["Model", "Results"])

,Model,Results
0,MultinomialNB,"(0.5, [[0, 10], [0, 15]])"
1,BernoulliNB,"(0.5, [[0, 10], [0, 15]])"
2,GaussianNB,"(1.0, [[10, 0], [0, 15]])"


### Question 4: How did you decide on the type of model to use? 

Binary classification tasks (e.g. spam/not spam) is fine with naive Bayes models.

GaussianNB seems to have the best performance of the 3 Naive Bayes models tested.

## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* of plagiarism and not just plagiarized or not.
* Utilize a different and larger dataset to see if this model can be extended to non-computer science, submitted answers.
* Use language or character-level analysis to find different and more features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.

These are all just options for extending your work. If you've completed all the tasks in this notebook and run your code, you've completed a real-world application, and can proceed to submit your work. Great job!